# Production Technology

The dataset contains `N = 441` firms observed over `T = 12` years, 1968-1979. There variables are: 
* `lcap`: Log of capital stock, $k_{it}$ 
* `lemp`: log of employment, $\ell_{it}$ 
* `ldsa`: log of deflated sales, $y_{it}$
* `year`: the calendar year of the observation, `year` $ = 1968, ..., 1979$, 
* `firmid`: anonymized indicator variable for the firm, $i = 1, ..., N$, with $N=441$. 

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import os
print(os.listdir())
import project_1 as est
from numpy import linalg as la
from scipy.stats import chi2
from scipy import stats
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [ ]:
dat = pd.read_csv('firms.csv')

# Descriptives

In [ ]:
dat.describe()

In [ ]:
#Descriptive statistics table
cols = ['lcap','lemp','ldsa']

tab = (
    dat[cols]
      .agg(['mean','kurt'])   # 'kurt' = excess kurtosis (normal = 0)
      .T
      .round(3)
      .rename(columns={'mean':'Mean', 'kurt':'Kurtosis'})
)

latex = tab.to_latex(
    index=True,
    caption='Beskrivende statistik (samlet): middelværdi og kurtosis',
    label='tab:desc_overall',
    float_format="%.3f",
    escape=False
)

print(latex)

In [ ]:
# De-log-transform variables 
dat['cap'] = np.exp(dat['lcap'])
dat['emp'] = np.exp(dat['lemp'])
dat['dsa'] = np.exp(dat['ldsa'])

cols = ['cap','emp','dsa']

tab = (
    dat[cols]
      .agg(['mean','kurt'])   # 'kurt' = excess kurtosis (normal = 0)
      .T
      .round(3)
      .rename(columns={'mean':'Mean', 'kurt':'Kurtosis'})
)

latex = tab.to_latex(
    index=True,
    caption='Beskrivende statistik (samlet): middelværdi og kurtosis',
    label='tab:desc_overall',
    float_format="%.3f",
    escape=False
)

print(latex)

In [ ]:
blue = "#5aa9e6"  # lys blå

fig = plt.figure(figsize=(10, 6), constrained_layout=True)
gs = fig.add_gridspec(nrows=2, ncols=2, height_ratios=[1.2, 1.0])

ax_top = fig.add_subplot(gs[0, :])                 # øverste (salg), span begge kolonner
ax_bl  = fig.add_subplot(gs[1, 0], sharey=ax_top)  # nederst venstre, deler y med top
ax_br  = fig.add_subplot(gs[1, 1], sharey=ax_top)  # nederst højre, deler y med top

# Øverst: Salg (centeret)
ax_top.hist(dat['ldsa'].dropna(), bins=30, color=blue, edgecolor="white", linewidth=0.5)
ax_top.set_title("Deflated Sales (log)", loc="center")

# Nederst venstre: Beskæftigelse
ax_bl.hist(dat['lemp'].dropna(), bins=30, color=blue, edgecolor="white", linewidth=0.5)
ax_bl.set_title("Employment (log)", loc="center")

# Nederst højre: Kapital
ax_br.hist(dat['lcap'].dropna(), bins=30, color=blue, edgecolor="white", linewidth=0.5)
ax_br.set_title("Adjusted Capital Stock (log)", loc="center")

# Fælles y-akse: kun én y-label på figuren
for ax in (ax_top, ax_bl, ax_br):
    ax.grid(True, alpha=0.25)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylabel("")              # ingen individuelle y-labels

fig.supylabel("Frequency")          # fælles y-label for alle paneler

# Skjul y-tick labels på højre nederste panel for mindre støj
ax_br.tick_params(labelleft=False)

# Ingen over-overskrift
try:
    fig._suptitle.remove()
except Exception:
    pass
plt.show()

In [ ]:
years = np.sort(dat['year'].unique())

# Blå palette: vælg et snævert interval af 'Blues' for harmoniske nuancer
cmap = plt.cm.Blues
colors = cmap(np.linspace(0.45, 0.90, len(years)))  # gør intervallet smallere/større efter smag
palette = {y: colors[i] for i, y in enumerate(years)}

fig, axes = plt.subplots(1, 2, figsize=(11, 5), sharey=True)

# Venstre: Employment (log) vs Adjusted Sales (log)
sns.scatterplot(
    x='lemp', y='ldsa', data=dat, ax=axes[0],
    hue='year', hue_order=years, palette=palette,
    edgecolor="white", s=28, alpha=0.85, legend=False
)
sns.regplot(
    x='lemp', y='ldsa', data=dat, ax=axes[0], scatter=False,
    color="#3a3a3a", ci=None, line_kws={'linewidth': 2}
)
axes[0].set_xlabel("Employment (log)")
axes[0].set_ylabel("")

# Højre: Capital (log) vs Adjusted Sales (log)
sns.scatterplot(
    x='lcap', y='ldsa', data=dat, ax=axes[1],
    hue='year', hue_order=years, palette=palette,
    edgecolor="white", s=28, alpha=0.85, legend=True
)
sns.regplot(
    x='lcap', y='ldsa', data=dat, ax=axes[1], scatter=False,
    color="#3a3a3a", ci=None, line_kws={'linewidth': 2}
)
axes[1].set_xlabel("Adjusted Capital Stock (log)")
axes[1].set_ylabel("")

# Kosmetik
for ax in axes:
    ax.grid(True, alpha=0.25)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    sns.despine(ax=ax)

fig.supylabel("Adjusted Sales (log)")
axes[1].legend(title="Year", bbox_to_anchor=(1.02, 1), loc="upper left", borderaxespad=0)

try:
    fig._suptitle.remove()
except Exception:
    pass

plt.tight_layout()
plt.show()


# Converting data to numpy format 

In [ ]:
dat.ldsa.values.shape

In [ ]:
#Define N and T for our models 
N = dat.firmid.unique().size
T = dat.year.unique().size
assert dat.shape[0] == N*T, f'Error: data is not a balanced panel'
print(f'Data has N={N} and T={T}')

In [ ]:
#Define sales as our dependent variable 
y = dat.ldsa.values.reshape((N*T,1))

#Laver et konstantled som repræsenterer det teknologiske niveau, A og definerer vores X matrix
ones = np.ones((N*T,1)) 
l = dat.lemp.values.reshape((N*T,1))
k = dat.lcap.values.reshape((N*T,1))
X = np.hstack([ones, l, k])
label_y = 'log_omsætning'
label_x = ['const', 'log_ansatte', 'log_kapital']

#Rename columns for easier reading
rename_dict = {'year': 'aar', 'ldsa':'log_omsætning', 'lemp': 'log_ansatte', 'lcap': 'log_kapital'}
dat = dat.rename(columns=rename_dict)

# OLS

In [ ]:
#Estimerer pooled ols for benchmark 
pooled_result = est.estimate(y, X, T=T)
est.print_table((label_y, label_x), pooled_result, title='Pooled OLS regression', floatfmt='.4f')

# FE

In [ ]:
###Transform the data#####
# Create transformation matrix
Q_T = np.eye(T) - (np.ones((T,T)))/T
#Print dimension of Q_T
print(f'Dimension of Q_T: {Q_T.shape}')

#Create y and x dot using the perm function
y_dot = est.perm(Q_T, y.reshape(-1, 1))           # (N*T, 1)
x_dot = est.perm(Q_T, X)                           # (N*T, K)
x_dot = x_dot[:,1:]

# Estimate the fe model using the estimate function
label_x_fe = ['log_ansatte', 'log_kapital'] 
fe_result = est.estimate(y_dot, x_dot, robust_se=True, transform='fe', T=T)
est.print_table((label_y, label_x_fe), fe_result, title="Fixed Effects", floatfmt='.4f')

# FD og serial correlation

In [ ]:
def fd_matrix(T):
    D_T = np.zeros((T-1, T))
    for i in range (T-1):
        D_T[i,i] = -1
        D_T[i, i+1] = 1
    return D_T

# Print the matrix
D_T = fd_matrix(T)

# use dt til at lave first differences
y_diff = est.perm(D_T, y.reshape(-1, 1))           # (N*(T-1), 1)
x_diff = est.perm(D_T, X)                           # (N*(T-1, K)

#udvælg kolonner
x_diff = x_diff[:, 1:]  # drop constant column

# Estimate FD 
fd_result = est.estimate(y_diff, x_diff, transform='fd', robust_se=True ,T=T)
label_x_fd = ['log_ansatte', 'log_kapital']

# Print results
est.print_table((label_y, label_x_fd), fd_result, title='FD regression', floatfmt='.4f')


In [ ]:
# Make function to calculate the serial correlation
def serial_corr(y, x, T):
    # Calculate the residuals
    if x.ndim == 1:
        x = x.reshape(-1, 1)     
    if y.ndim == 1:
        y = y.reshape(-1, 1)                     # sikr 2D
    b_hat = la.inv(x.T @ x) @ (x.T @ y)
    e = y - x @ b_hat                         # (N*T,)

    # Create a lag transformation matrix (perm to lag)
    # Picks [y1, y2, ..., y_{T-1}]
    P_lag = np.eye(T-1, T)                    # (T-1 x T)

    # Lag residuals panelvis
    E = e.reshape(-1, T)                      # (N, T)
    e_l = (E @ P_lag.T).reshape(-1, 1)        # (N*(T-1), 1)  ← regressor

    # Create a transformation matrix that removes the first time-period (perm to drop first)
    # Picks [y2, y3, ..., y_T]
    I_T = np.eye(T-1, T, k=1)                 # (T-1 x T)

    # Remove first observation of each individual (align "now" with the lag)
    e = (E @ I_T.T).reshape(-1,1)              # (N*(T-1),)    ← dependent var

    # Calculate the serial correlation (no intercept in FD)
    N = e.shape[0] // (T-1)
    return est.estimate(e, e_l, transform='fd', T=T-1)

In [ ]:
# Estimate serial correlation
corr_result = serial_corr(y_diff, x_diff, T-1)
# Print results
label_ye = 'OLS residual, e\u1d62\u209c'
label_e = ['e\u1d62\u209c\u208B\u2081']
est.print_table(
    (label_ye, label_e), corr_result, 
    title='Serial Correlation', floatfmt='.4f'
)
# calculate p-value for test of no serial correlation
r = corr_result['b_hat'][0]
p_val = 2 * (1 - stats.norm.cdf(np.abs(r) / corr_result['se'][0]))
#p_val = float(p_val)
print(f'P-value for test of no serial correlation: {p_val[0]:8.4f}')

## Test for strict exo 

In [ ]:
#Add a lead of the emp variable,
# Create F_T to shift the emp variable one period forward
F_T = np.eye(T, T, k=1)
# Create emp_lead_full using perm function
emp_lead_full = est.perm(F_T, X[:, 2].reshape(-1, 1)) # the shape is (N*T, 1)
# Remove the last observed year for every individual
I_T = np.eye(T-1, T)
# Remove the last observations
x_exo = est.perm(I_T, X)                       # (N*(T-1), K)
y_exo = est.perm(I_T, y.reshape(-1, 1))        # (N*(T-1), 1)
emp_lead = est.perm(I_T, emp_lead_full)             # (N*(T-1), 1)
assert x_exo.shape[0] == emp_lead.shape[0] == y_exo.shape[0]

In [ ]:
# Add union_lead to x_exo
x_exo = np.hstack((x_exo, emp_lead))                 # (N*(T-1), K+1)

# Within transform the data (demean over T-1)
Tm1 = T - 1
Q_T = np.eye(Tm1) - np.ones((Tm1, Tm1)) / Tm1          # (T-1)×(T-1)

# apply panel-wise with your perm helper
yw_exo = est.perm(Q_T, y_exo.reshape(-1, 1)).reshape(-1,)   # (N*(T-1),)
xw_exo = est.perm(Q_T, x_exo)                               # (N*(T-1), K+1)

# Select variables (employement, capital stock, emp_lead)
xw_exo = np.hstack((xw_exo[:, 1:3], xw_exo[:, -1].reshape(-1, 1)))

In [ ]:
# Estimate model
yw_exo = yw_exo.reshape(-1, 1)
exo_test = est.estimate(yw_exo, xw_exo, T=T-1, robust_se=True, transform='fe')

# Print results
label_exo = ['emp'] + ['cap'] + ['emp lead']
est.print_table((label_y, label_exo), exo_test, title='Exogeneity test', floatfmt='.4f')

In [ ]:
#Test if variable 5 is significant different from 0
from scipy import stats
b = exo_test['b_hat']
cov = exo_test['cov']
se = np.sqrt(np.diag(cov))
t_stat = b[2] / se[2]
# frihedsgrader (hvis ikke df_resid findes, så udled dem)
df = exo_test.get('df_resid', len(yw_exo) - len(b))
p_value = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=3))
p_value = float(p_value)  # or p_value = p_value.item()
print(f'Test statistic for H0: beta_3 = 0 is {t_stat} with p-value {p_value:8.4f}')  

## WEAK EXO

In [ ]:
# --- Build selector to pick levels at time t for rows t=2..T (aligns with Δy_t) ---
R_T = np.eye(T-1, T, k=1)                      # (T-1)×T, selects columns 2..T

# Levels at time t (drop constant to mirror x_diff)
x_level_t = est.perm(R_T, X)[:, 1:]            # (N*(T-1), K-1) -> same vars as in x_diff

# Assemble augmented RHS: [Δk, Δl, k_t, l_t]
X_fd_exog = np.hstack([x_diff, x_level_t[:, :x_diff.shape[1]]])

# Labels (adjust names if needed)
labels_fd_exog = ['Δlog_ansatte', 'Δlog_kapital', 'log_ansatte_t', 'log_kapital_t']

# Estimate FD regression with levels included
fd_exog_res = est.estimate(y_diff, X_fd_exog, transform='fd', T=T-1, robust_se=True)

In [ ]:
b = fd_exog_res['b_hat']
cov = fd_exog_res['cov']
se = np.sqrt(np.diag(cov))
t_stat = b[2] / se[2]
# frihedsgrader (hvis ikke df_resid findes, så udled dem)
df = fd_exog_res.get('df_resid', len(y_diff) - len(b))
p_value = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=3))
p_value = float(p_value)  # or p_value = p_value.item()
print(f'Test statistic for H0: beta_3 = 0 is {t_stat} with p-value {p_value:8.4f}')  

## Hypothesis test

In [ ]:
b_hat = fe_result['b_hat']
cov   = fe_result['cov']

ansatte_index = label_x.index('log_ansatte')
kapital_index = label_x.index('log_kapital')

# Gør vektorer/matricer klar
b = np.asarray(b_hat).reshape(-1)   # (P,)
V = np.asarray(cov)                 # (P,P)
P = b.size
if V.shape != (P, P):
    raise ValueError(f"Kovarians har forkert form: {V.shape}, forventede {(P,P)}")

# Byg R (1 x P) med 1 på de to relevante koefficienter
R = np.zeros((1, P))
R[0, [ansatte_index, kapital_index]] = 1.0

# Wald-test for H0: beta_kapital + beta_ansatte = 1
num = float(R @ b - 1.0)           # c'β - q, q=1
den = float(R @ V @ R.T)           # c' V c
if not np.isfinite(den) or den <= 0:
    raise ValueError("Variansen af restriktionen er ikke positiv/finitsk; tjek kovarians og indeks.")

wald = (num ** 2) / den
p_value = stats.chi2.sf(wald, df=1)

print(f"Wald test statistic: {wald:.4f}")
print(f"p-value: {p_value:.4g}")